In [1]:
!pip install datasets torch

In [3]:
!pip install transformers==4.30.0

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [49 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-cpython-312\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
  creating build\lib.win-amd64-cpython-312\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
  creating build\lib.win-amd64-cpython-312\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
  creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  creating build\lib.win-amd64-c

In [15]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
import os

In [21]:
class ModelTrainer:
    def __init__(self):
        self.train_path = os.path.join("../artifacts","data_transformation","train")
        self.evaluate_path = os.path.join("../artifacts","data_transformation","evaluate")
        self.model_name = "google/flan-t5-base"
        self.output_path = os.path.join("../artifacts","model_trainer")
        self.evaluation_strategy='no'
        self.learning_rate=2e-5
        self.per_device_train_batch_size=4
        self.per_device_eval_batch_size=4
        self.num_train_epochs=3
        self.weight_decay=0.01

    def initiate_training(self):
        tokenizer = T5Tokenizer.from_pretrained(self.model_name)
        model = T5ForConditionalGeneration.from_pretrained(self.model_name)
        
        tokenized_dataset_train = load_from_disk(self.train_path)
        tokenized_dataset_evaluate = load_from_disk(self.evaluate_path)

        if not os.path.exists(self.output_path):
            os.makedirs(self.output_path)
        
        training_args = TrainingArguments(
            output_dir=self.output_path,
            evaluation_strategy=self.evaluation_strategy,  
            learning_rate=self.learning_rate,
            per_device_train_batch_size=self.per_device_train_batch_size,
            per_device_eval_batch_size=self.per_device_eval_batch_size,
            num_train_epochs=self.num_train_epochs,
            weight_decay=self.weight_decay
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset_train,
            eval_dataset=tokenized_dataset_evaluate
        )

        trainer.train()

        model.save_pretrained(os.path.join(self.output_path, "fine_tuned_flan_t5_base_model"))
        tokenizer.save_pretrained(os.path.join(self.output_path, "fine_tuned_flan_t5_base_tokenizer"))

In [19]:
model_trainer = ModelTrainer()
model_trainer.initiate_training()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

C:\Users\USER\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

KeyboardInterrupt: 